### PRUEBA CON 30 TWEETS DE LA API, PARA PROBAR EL ENTRENAMIENTO DEL MODELO CON POCOS DATOS.


In [16]:
import pandas as pd
import re
from transformers import RobertaTokenizer
import toml
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer
import torch
from transformers import RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader
import time
from sklearn.metrics import accuracy_score
from deep_translator import GoogleTranslator
import torch.nn as nn

In [37]:
translator = Translator()
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20

In [33]:
#df_prueba_roberta = pd.read_csv('C:/Users/Agustín/Desktop/4Geeks/Clases/30. Proyecto Final/Public Environment/Final-Project---Luis-Augustin-Ale/notebooks/aux_prueba.csv')
#df_prueba_roberta = df_prueba_roberta.drop(columns=['Unnamed: 0'])
df_prueba_roberta.rename(columns={'sentiment': 'tweets'}, inplace=True)
df_prueba_roberta

,0,tweets
0,Sat Oct 05 15:00:34 +0000 2024,Now you can see the streaming of Premodern Mad...
1,Sat Oct 05 15:00:34 +0000 2024,@FedeeValver Madrid will concede
2,Thu Sep 19 01:00:00 +0000 2024,Celebrate Prof. Oscar Garcia-Prada's 60th birt...
3,Sat Oct 05 15:00:31 +0000 2024,@Steve_Labile Frero arretez de lui trouver une...
4,Sat Oct 05 15:00:30 +0000 2024,"⚽️💥 Back Inter Milan, Real Madrid and Sporting..."
5,Sat Oct 05 15:00:29 +0000 2024,@Alvaro_varito Florentino cada vez pasa más fr...
6,Sat Oct 05 15:00:28 +0000 2024,@GxlDeMessi18 Penal para el real de Madrid
7,Tue Oct 01 13:01:57 +0000 2024,Unlock your best self with fasting! 🍽️💪 Burn f...
8,Sat Oct 05 15:00:24 +0000 2024,@madrid_total2 Como me encanta que sin estar e...
9,Sat Oct 05 15:00:22 +0000 2024,Hoy nos pita el árbitro del Frente Atlético. h...


In [38]:
'''def traducir(text):
    # Detectar el idioma automáticamente y traducirlo al inglés
    if GoogleTranslator(source='auto', target='en').translate(text):
        return GoogleTranslator(source='auto', target='en').translate(text)
    else:
        return text

df_prueba_roberta['tweets_traducidos'] = df_prueba_roberta['tweets'].apply(traducir)'''
df_prueba_roberta.head()


,0,tweets,tweets_traducidos
0,Sat Oct 05 15:00:34 +0000 2024,Now you can see the streaming of Premodern Mad...,Now you can see the streaming of Premodern Mad...
1,Sat Oct 05 15:00:34 +0000 2024,@FedeeValver Madrid will concede,@FedeeValver Madrid will concede
2,Thu Sep 19 01:00:00 +0000 2024,Celebrate Prof. Oscar Garcia-Prada's 60th birt...,Celebrate Prof. Oscar Garcia-Prada's 60th birt...
3,Sat Oct 05 15:00:31 +0000 2024,@Steve_Labile Frero arretez de lui trouver une...,@Steve_Labile Brother stop making excuses for ...
4,Sat Oct 05 15:00:30 +0000 2024,"⚽️💥 Back Inter Milan, Real Madrid and Sporting...","⚽️💥 Back Inter Milan, Real Madrid and Sporting..."


In [39]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

c:\Users\Agustín\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [49]:
tweets = df_prueba_roberta['tweets_traducidos'].to_list()
type(tweets)

list

In [50]:
encoded_tweet = tokenizer.encode_plus(tweets, add_special_tokens=True,                      # Adds [CLS] and [SEP] 
                                      max_length=128,                                       # Max sequence length 
                                      padding='max_length',                                 # Pad to max_length 
                                      truncation=True,                                      # Truncate if the text is too long 
                                      return_attention_mask=True,                           # Return attention mask 
                                      return_tensors='pt')                                  # Return PyTorch tensors

In [52]:
print(encoded_tweet['input_ids'])
print(encoded_tweet['attention_mask'])

tensor([[0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])


In [53]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

c:\Users\Agustín\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Agustín\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at

### We freeze all layers and then unfreeze the last 2 layers. After we declare that layers from 12th forward will be trained

In [ ]:
for layer in model.roberta.encoder.layer[:10]:
    for param in layer.parameters():
        param.requires_grad = False

# Unfreeze the last 2 layers
for layer in model.roberta.encoder.layer[10:12]:
    for param in layer.parameters():
        param.requires_grad = True

# Ensure layers after the 12th are trainable
for layer in model.roberta.encoder.layer[12:]:
    for param in layer.parameters():
        param.requires_grad = True

In [55]:
### TRADUCIRRRRRRRRRRRRRRRRRRRRRRRRRRRRR

# We declare a new class to load the ROBERTA model and add two personalized layers
class CustomRobertaModel(nn.Module):
    def __init__(self, num_labels=2):
        super(CustomRobertaModel, self).__init__()
        # We load the pretrained Roberta model base 
        self.roberta = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)
        
        # Agregamos capas adicionales densas (fully connected) 
        self.additional_layer_1 = nn.Linear(self.roberta.config.hidden_size, 512)                   # Capa adicional 1 - Realiza una transformacion lineal de la entrada 
        self.additional_layer_2 = nn.Linear(512, 256)                                               # Capa adicional 2
        self.classifier = nn.Linear(256, num_labels)                                                # Capa final de clasificación (ajusta el tamaño según tu tarea)
        
        # Función de activación
        self.relu = nn.ReLU()
    
    def forward(self, input_ids, attention_mask=None, labels=None):
        # Pasamos los datos a través del modelo de RoBERTa
        outputs = self.roberta.roberta(input_ids=input_ids, attention_mask=attention_mask)
        
        # Obtenemos el embedding final (última capa oculta de RoBERTa)
        pooled_output = outputs[1]  # La salida de RoBERTa es una tupla: [0] son las logits, [1] el embedding
        
        # Pasamos por las capas adicionales
        x = self.relu(self.additional_layer_1(pooled_output))
        x = self.relu(self.additional_layer_2(x))
        
        # Capa final de clasificación
        logits = self.classifier(x)
        
        # Si se proporcionan etiquetas, calculamos la pérdida
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
            return loss, logits
        else:
            return torch.sigmoid(x)                         # previo code de esta linea era : return logits

# Cargar el modelo personalizado
model_custom = CustomRobertaModel(num_labels=2)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Move the model to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_custom.to(device)


In [ ]:
# COMO DICE CHATGPT QUE DEBERIA SEGUIR LA CODIFICACION, ADAPTAR
'''
# Asegurar que todos los parámetros sean entrenables
for param in model.parameters():
    param.requires_grad = True

# Definir optimizador
optimizer = AdamW(model.parameters(), lr=2e-5)  # Ajusta el lr si es necesario

# Configurar el entrenamiento
epochs = 3
model.train()  # Asegurar que el modelo está en modo entrenamiento

for epoch in range(epochs):
    running_loss = 0.0
    for batch in train_loader:
        # Mover los datos al dispositivo
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Reiniciar gradientes
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass y optimización
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {running_loss/len(train_loader)}")
'''